In [1]:
# !pip install sentence_transformers
# !git clone https://github.com/flipz357/S3BERT
# !wget https://www.cl.uni-heidelberg.de/~opitz/data/s3bert_all-mpnet-base-v2.tar.gz
# !wget https://www.cl.uni-heidelberg.de/~opitz/data/config_s3bert_all-mpnet-base-v2.py
# !tar -xzvf s3bert_all-mpnet-base-v2.tar.gz
# !mv config_s3bert_all-mpnet-base-v2.py config.py
# !pip install --upgrade pip
# !pip install --upgrade openai
# !pip install --upgrade python-dotenv
# !pip install catboost retry llama-index polars

In [27]:
import pandas as pd
from pathlib import Path
from table_understanding.context_sample import *
from tqdm.auto import tqdm
import json
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import os
import signal
from ast import literal_eval
#https://scikit-learn.org/stable/auto_examples/cluster/plot_optics.html

In [3]:
import ast
import numpy as np
from sentence_transformers import SentenceTransformer
import config
import S3BERT.src.prediction_helpers as ph
import torch
from operator import itemgetter

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
# get decomposed model
model = SentenceTransformer("./" + config.SBERT_SAVE_PATH + "/", device=device)

cuda


In [5]:
def s3bert_preds(xsent_encoded, ysent_encoded):
  # get similarity scores of different features
  preds = ph.get_preds(xsent_encoded, ysent_encoded, biases=None, n=config.N, dim=config.FEATURE_DIM)

  # print similarity scores of different features
  features = ["global"] + config.FEATURES[2:] + ["residual"]
  jl_save = {}
  for i, x in enumerate(xsent_encoded):
      sims = preds[i]
      jl = {k:v for k,v in zip(features, sims)}
  return jl

def s3bert_encode(xsent):
    return model.encode(xsent)

In [6]:
def handler(signum, frame):
   raise Exception("end of time")
signal.signal(signal.SIGALRM, handler)

<Handlers.SIG_DFL: 0>

In [7]:
NYC_PATH = Path("/scratch/bf996/datasets/socrata/data/data.cityofnewyork.us")
TARGET_PATH = Path("/scratch/bf996/datasets/socrata/nyc_context.csv")
nyc_dfs = list(NYC_PATH.rglob("**/*.csv"))
nyc_metadata = list(NYC_PATH.rglob("**/*.json"))

In [8]:
len(nyc_dfs)

2789

In [11]:
context_length=10

## Main Loop for Generating Context Dataframe

In [12]:
f_path = []
m_path = []
md_jsons = []
sample_dicts = []
sample_encodings = []
for f, m in tqdm(zip(nyc_dfs, nyc_metadata)):
    n = 2000000
    size = os.path.getsize(f)
    #TODO: large DFS
    if size > 100000000:
        print(f"skipping {f} \n")
        continue
    try:
        signal.alarm(40)
        chunks = pd.read_csv(f, chunksize=n)
        df = next(chunks)
        signal.alarm(0)
    except Exception as e:
        print(f"Exception {e} for dataframe {f}")
        continue
    df = df.dropna(axis=1, how='all')
    if len(df) == 0:
        print(f"No samples found for dataframe {f} \n")
        continue
    sample_dict = {}
    try:
        signal.alarm(40)
        sample_dict = get_df_sample(df, len_context=context_length, full=True)
        fname = str(f.name).replace(".", "_")
        spath = "/scratch/bf996/datasets/socrata/encodings/" + fname + ".csv"
        sample_dict.to_csv(spath)
        sample_dicts.append(spath)
        signal.alarm(0)
        sample_encoding = {}
        for col in sample_dict.columns:
            sample_str = " ".join(str(k) for _, k in sample_dict[col].items())
            enc = s3bert_encode([sample_str])
            colnm = col[:12].replace(" ", "_").replace("/", "")
            fpath = "/scratch/bf996/datasets/socrata/encodings/" + fname + "_" + colnm + ".npy"
            fn = open(fpath, 'wb')
            np.save(fn, enc)
            fn.close()            
            sample_encoding[fname + "_" + colnm] = fpath
    except Exception as e:
        print(f"Exception {e} for dataframe {f}")
        continue    
    f_path.append(f)
    m_path.append(m)
    with open(m, 'r', encoding='utf-8') as alt_f:
      md_jsons.append(json.load(alt_f))
    #TODO: metadata embedding
#       try:
#         sample_encodings["metadata"] = s3bert_encode([md_jsons['name'] + " " + md_jsons['description']])
#       except:
#         pass
    sample_encodings.append(sample_encoding)


0it [00:00, ?it/s]

Exception single positional indexer is out-of-bounds for dataframe /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.kkng-ugna.rows.csv


6it [00:10,  1.10it/s]

skipping /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.7ym2-wayt.rows.csv 

Exception No columns to parse from file for dataframe /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.rw6k-9v7e.rows.csv


17it [00:30,  1.21it/s]

...

Exception single positional indexer is out-of-bounds for dataframe /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.vg63-xw6u.rows.csv


2784it [47:31, 10.33s/it]

Exception end of time for dataframe /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.myei-c3fa.rows.csv


2789it [47:33,  1.02s/it]

Exception single positional indexer is out-of-bounds for dataframe /scratch/bf996/datasets/socrata/data/data.cityofnewyork.us/data-cityofnewyork-us.35sw-rdxj.rows.csv


In [13]:
context_df = pd.DataFrame({"csv_path" : f_path,
                          "meta_path" : m_path,
                           "metadata" : md_jsons,
                           "context" : sample_dicts,
                           "encodings" : sample_encodings,
                          })

context_df.to_csv(TARGET_PATH)

In [18]:
len(context_df)

2224

## Generate Similarity Matrix

In [28]:
NYC_SIM_MAT_PATH = Path("/scratch/bf996/datasets/socrata/data/NYC_col_sim_matrix.json")
NYC_SIM_MAT_PATH.touch(exist_ok=True)

In [29]:
context_df = pd.read_csv(TARGET_PATH)

In [30]:
similarity_matrix = {}

In [ ]:
for i in tqdm(range(len(context_df))):
    d1 = literal_eval(context_df.loc[i, 'encodings'])
    for j in tqdm(range(i, len(context_df)), leave=False):
        sim_list = {}
        d2 = literal_eval(context_df.loc[j, 'encodings'])
        #t1 = pd.read_json(context_df.loc[i, 'context'], orient='records')
#         print("core dataframe")
#         print(t1)
        #t2 = pd.read_json(context_df.loc[j, 'context'], orient='records')
#         print("comparison dataframe")
#         print(t2)
        x_sents={}
        y_sents={}
        for k, v in d1.items():
            with open(v, 'rb') as rdf:
                x_sents[k]=np.load(rdf)
        for k, v in d2.items():
            with open(v, 'rb') as rdf:
                y_sents[k]=np.load(rdf)
        for xsk, xsv in x_sents.items():
            for ysk, ysv in y_sents.items():
                if xsk == ysk:
                    continue
                predv = s3bert_preds(xsv,ysv)
                similarity_matrix[xsk + "_" + ysk]=predv
#                 print(f"results for {xsk} and {ysk} \n")
#                 print(f"context for x was {tsv}")
#                 print(f"similarity was {predv}")
        with open(NYC_SIM_MAT_PATH, 'w', encoding='utf-8') as my_f:
            json.dump(similarity_matrix, my_f, ensure_ascii=False, indent=4)
print(f"Similarity matrix assembled! Size was {len(similarity_matrix)}")
with open(NYC_SIM_MAT_PATH, 'w', encoding='utf-8') as my_f:
    json.dump(similarity_matrix, my_f, ensure_ascii=False, indent=4)


  0%|          | 0/2224 [00:00<?, ?it/s]

  0%|          | 0/2224 [00:00<?, ?it/s]

## Clustering Sentence Embeddings

In [ ]:
#https://blog.eduonix.com/artificial-intelligence/clustering-similar-sentences-together-using-machine-learning/

In [ ]:
from gensim.models import Word2Vec
import nltk
import numpy as np
from sklearn. cluster import KMeans
from sklearn import cluster
from sklearn import metrics
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering

In [ ]:
#Let X be a list of vectors

In [ ]:
import matplotlib.pyplot as plt
wcss = []
for i in range (1,4):
    kmeans=KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,4), wcss)
plt.title( 'The Elbow Method')
plt.xlabel ('Number of clusters')
plt.ylabel ('WCSs')
plt.show()

In [ ]:
#n_clusters = ELBOW
clf = KMeans(n_clusters=n_clusters,
            max_iter=100,
            init='kmeans++',
            n_init=1)
labels = clf.fit_predict(X)
for index, sentence in enumerate(sentences):
    print(str(labels[index]) + ":" + str(sentence))

## Gather Meta-Context, Annotate with Properties (CPA)

In [ ]:
"""
1. Sample ~100x from each label cluster
2. Assemble into a mega-sample
3. Send to ChatGPT with a list of possible CPA annotations
4. Apply annotations to cluster
5. Generate cluster description based on annotations and/or summarizing metadata
"""

## Create LLAMA Training Dataset

In [ ]:
NYC_LLAMA_TRAIN = Path("/scratch/bf996/datasets/socrata/data/NYC_llama_training.json")
instances = {}
col_to_row = context_df.columns
for idx, row in context_df.iterrows():
    context = literal_eval(row[3])
    metadata = literal_eval(row[4])
    for k,v in context.items():
        cont_str = " ".join(str(s) for _, s in v.items())
        instance = {"Instruction" : "Select the category which best matches the input. \n", "Input" : cont_str, "Output" : k+"\n"}
        #instance = {"Instruction" : "Describe the input in . \n", "Input" : , "Output" : metadata+"\n"}
        instances[row[0]+"_"+col_nm] = instance

In [ ]:
df = pd.DataFrame.from_dict(data, instances='index')
df.to_json(NYC_LLAMA_TRAIN, orient='records', indent=4)